# Library

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.error import HTTPError

import time
import datetime
from datetime import datetime, timedelta

import re

import json

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import sys

%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# pathes
art_project_path = '/content/drive/MyDrive/Art/Art_Project'
art_project_path_backups = '/content/drive/MyDrive/Art/Art_Project/parsing_backups'
art_project_path_tests = '/content/drive/MyDrive/Art/Art_Project/parsing_tests'

# files
page_s = '/page_parsing_status.csv'

## Sites' list for parsing

In [4]:
urls_news = ['https://www.theartnewspaper.ru/', 'https://artinvestment.ru/', 'http://artuzel.com/'] # 'https://artguide.com/', 'http://artuzel.com/', 
# https://artguide.com/ - <div class="chromeframe">
#    Ваш браузер <em>устарел!</em> <a href="http://browsehappy.com/">Установите другой браузер</a> или
#    <a href="http://www.google.com/chromeframe/?redirect=true">Google Chrome Frame</a>, чтобы просматривать этот сайт.
urls_datasets = ['https://artchive.ru/', 'https://artinvestment.ru/', 'http://artuzel.com/']
urls_archives = ['https://artchive.ru/', 'https://russianartarchive.net/ru'] # есть API - https://api.raan.garagemca.org/documentation/#/
urls_fairs = ['https://www.cosmoscow.com/ru/']

# Unused code

In [ ]:
# payload = {'key1': 'value1', 'key2': ['value2', 'value3']}
# r = requests.get('https://httpbin.org/get', params=payload)
# r.json()

In [ ]:
# print(artinvest_page.prettify())

# Functions

## Data mining
1. __get_page(url)__ -> bs_page - Collect request status

2. Files' functions:
- 2.1 __open_file_csv(file, folder)__ -> df - Open file.csv
- 2.2 __save_df_file(df, file, folder)__ -> - Save df to csv file

In [5]:
# 1. Collect request status - saving every get result

def get_page(url):
  start_time = time.time()

  response = requests.get(url, headers={'User-Agent': 'Chrome/97.0.4692.71'}) # timeout=0.001
# Edge/97.0.1072.55
# Chrome/97.0.4692.71
  try:
    response.raise_for_status()

  except requests.exceptions.HTTPError: # for correction
    sys.exit(print('HTTPError: ' + str(url)))

#except (HTTPError, MissingSchema): 
# 503 Server Error: Service Temporarily Unavailable for url: https://artinvestment.ru/news/artnews/ 
# Invalid URL

  else:
    get_page_stat_new = pd.DataFrame({'page_url': [url],
                                      'status_code': [response.status_code],
                                      'encoding': [response.encoding],
                                      'raise_for_status': [response.raise_for_status()],
                                      'parsing_date': [datetime.now().date()],
                                      'parsing_time': [datetime.now().time()],
                                      'parsing_sec_speed': [time.time() - start_time]})

    bs_page = BeautifulSoup(response.text, 'html.parser')
  
    get_page_stat = open_file_csv(page_s, 'live') # 2. Open file.csv
    get_page_stat = pd.concat([get_page_stat, get_page_stat_new])
    save_df_file(get_page_stat, page_s, 'live') # 3. Save df to csv file

    return bs_page

In [6]:
# 2.1 Open file.csv
def open_file_csv(file, folder): # folder: live, backup or test
  if folder=='live':
    df = pd.read_csv(art_project_path + file)
  elif folder=='backup':
    df = pd.read_csv(art_project_path_backups + file)
  elif folder=='test':
    df = pd.read_csv(art_project_path_tests + file)
  return df

# 2.2 Save df to csv file
def save_df_file(df, file, folder): # folder: live, backup or test
  if folder=='live':
    df.to_csv(art_project_path + file, index=False)
  elif folder=='backup':
    df.to_csv(art_project_path_backups + file, index=False)
  elif folder=='test':
    df.to_csv(art_project_path_tests + file, index=False)

In [74]:
# Date transfomation
def date_transfomation(cell):
  try:
    cell = pd.to_datetime(datetime.strptime(cell, '%d.%m.%Y').strftime('%Y-%m-%d')).date()
  except ValueError:
    cell = pd.to_datetime(cell).date()
  return cell

## Navigation tree
3. __save_parsing_dict(parsing_dict)__ -> - Save dict with sites for parsing:
- 3.1 with sites for parsing
- 3.2 with the last navigation tree
4. __open_parsing_dict()__ -> parsing_dict - Open dict with sites for parsing:
- 4.1 with sites for parsing
- 4.2 with the last navigation tree
5. __check_tree(file, new_tree)__ -> - Checking for updates in navigation tree ***(!!! not for tanp)***

In [15]:
current_dict = '/sites_for_parsing.txt'
artinvestment_tree = '/artinvestment_current_tree.txt'
theartnewspaper_tree = '/theartnewspaper_current_tree.txt'

tanp_total_tree = '/theartnewspaper_total_tree.txt'
artinvest_total_tree = '/artinvestment_total_tree.txt'

# 3. Save dict:
# 3.1 with sites for parsing
def save_parsing_dict(parsing_dict):
  with open(art_project_path + current_dict, 'w') as outfile:
    json.dump(parsing_dict, outfile)

# 3.2 with the last navigation tree
def save_dict(tree_dict, file):
  with open(art_project_path + file, 'w') as outfile:
    json.dump(tree_dict, outfile)

# 4. Open dict:
# 4.1 with sites for parsing
def open_parsing_dict():
  with open(art_project_path + current_dict, 'rb') as infile:
    parsing_dict = json.load(infile)
  return parsing_dict

# 4.2 with the last navigation tree
def open_dict(file):
  with open(art_project_path + file, 'rb') as infile:
    tree_dict = json.load(infile)
  return tree_dict

In [8]:
# 5. Checking for updates in navigation tree
def check_tree(file, new_tree):
  current_tree = open_dict(file) # 4.2 with the last navigation tree
  in_old_tree_dict = {}
  in_new_tree_dict = {}
  for key in current_tree: # str format
    if (int(key) in new_tree and current_tree[key] != new_tree[int(key)]):
      in_old_tree_dict[int(key)] = current_tree[key] # int format
    elif int(key) not in new_tree:
      in_old_tree_dict[int(key)] = current_tree[key] # int format
  for key in new_tree: # int format
    if (key not in in_old_tree_dict and str(key) in current_tree and new_tree[key] != current_tree[str(key)]):
      in_new_tree_dict[key] = new_tree[key] # int format
    elif (key not in in_old_tree_dict and str(key) not in current_tree):
      in_new_tree_dict[key] = new_tree[key] # int format

  if len(current_tree) == len(new_tree) and len(in_old_tree_dict) == 0 and len(in_new_tree_dict) == 0:
    result = print('Nothing has changed')
  else:
    result = print('Length of navigation three has changed - {}\nChanges in the old tree are: {}\nChanges in the new tree are: {}'.format(len(current_tree)!=len(new_tree), in_old_tree_dict, in_new_tree_dict))

  return result

# artinvestment.ru
- nav tree contains not only articles' srctions, but a lot if extra info 
- create a logic how to choose and check news and articles

## Navigation tree

In [9]:
# Collect page data
artinvest_url = urls_news[1]
artinvest_page = get_page(artinvest_url) # 1. Collect request status
artinvest_page.title.string

'Цены на картины'

In [50]:
# Collect topics for parsing

def get_topic_dict(nav_tree):
  list_titles = []
  list_links = []
  #count_list = []
  #count = 1
  for a in nav_tree.find_all('a'):
    if a.get('title') is not None:
      if str(a.get('title')) not in list_titles:
        list_titles.append(str(a.get('title')))
        #count_list.append(count)
        #count = count+1
        list_links.append(str(a.get('href'))) # , str(a.get('title'))
    else:
      if a.get_text() is not None:
        if str(a.get_text()) not in list_titles:
          list_titles.append(str(a.get_text()))
          #count_list.append(count)
          #count = count+1
          list_links.append(str(a.get('href'))) # , str(a.get_text())

  #tree_dict = {key: value for key, value in zip(count_list, list_links)}
  return list_titles, list_links

In [60]:
# Collect topics for parsing from navigation tree
artinvest_tree = artinvest_page.find('nav', id='tree')
curr_list_titles, curr_list_links = get_topic_dict(artinvest_tree)
print('curr_list_titles length is {}'.format(len(curr_list_titles)))

# finding what is not in current total tree dict
artinvest_curr_total_tree = open_dict(artinvest_total_tree)

artinvest_total_list_old = []
for k1, v1 in artinvest_curr_total_tree.items():
  for k2, v2 in v1.items():
    artinvest_total_list_old.append(v2[1])
print('artinvest_total_list_old length is {}'.format(len(artinvest_total_list_old)))

new_not_in_old_list = list(set(curr_list_titles) - set(artinvest_total_list_old))
print('count of titles that is not in the current total tree is {}'.format(len(new_not_in_old_list)))

count_list = []
count = len(artinvest_total_list_old)
for t in new_not_in_old_list:
  count += 1
  count_list.append(count)

add_links_list = []
for t in new_not_in_old_list:
  if t in curr_list_titles:
    add_links_list.append(curr_list_links[curr_list_titles.index(t)])
  else:
    add_links_list.append('-')

list_links = []
for l in add_links_list:
  list_links.append([l, new_not_in_old_list[add_links_list.index(l)]])

add_dict = {key: value for key, value in zip(count_list, list_links)}
add_dict

curr_list_titles length is 48
artinvest_total_list_old length is 47
count of titles that is not in the current total tree is 1


{48: ['https://artinvestment.ru/invest/new_names/', 'Новые имена']}

In [61]:
for k1, v1 in artinvest_curr_total_tree.items():
  print(k1)

prob_for_pars_articles
prob_for_pars_ds_other
def_not_for_pars


In [88]:
prob_for_pars_articles_dict = artinvest_curr_total_tree['prob_for_pars_articles']

for k, v in add_dict.items():
  prob_for_pars_articles_dict.update({str(k): v})
len(prob_for_pars_articles_dict)

19

In [64]:
# updating the current total tree
artinvest_curr_total_tree.update({'prob_for_pars_articles': prob_for_pars_articles_dict})

artinvest_total_list_old = []
for k1, v1 in artinvest_curr_total_tree.items():
  for k2, v2 in v1.items():
    artinvest_total_list_old.append(v2[1])
len(artinvest_total_list_old)

48

In [87]:
save_dict(artinvest_curr_total_tree, artinvest_total_tree) # 3.2 with the last navigation tree

### logic of choosing articles category

In [101]:
list_links = []
list_titles = []

for k, v in prob_for_pars_articles_dict.items():
  list_links.append(v[0])
  list_titles.append(v[1])

records_total_list = []
pages_total_list = []
records_max_list = []
records_value_list = []
last_article_date_list = []

for link in list_links: # page with news list of certain category
  page = get_page(link) # 1. Collect request status

  # counter block of the page with news list
  table_blocks = page.find_all('div', class_='pager')

  records_total_list.append(int(table_blocks[0].find_all('em')[0].text))
  pages_total_list.append(int(table_blocks[0].find_all('em')[1].text))
  records_max_list.append(int(table_blocks[0].find('input').get('title').split(' ')[1]))
  records_value_list.append(int(table_blocks[0].find('input').get('value')))

  # current page data block of the page with news list
  list_advices = page.find_all('ul', class_='list advices')

  em = list_advices[0].find_all('em')[0]
  last_article_date_list.append(em.text)

df_counter = pd.DataFrame({'page_url': list_links,
                           'page_title': list_titles,
                           'records_total': records_total_list,
                           'pages_total': pages_total_list,
                           'records_max': records_max_list,
                           'records_value': records_value_list,
                           'last_article_date': last_article_date_list
                           })
df_counter['last_article_date'] = df_counter['last_article_date'].dropna().apply(lambda x: date_transfomation(x).year)

curr_total_title_list = df_counter.loc[df_counter['last_article_date']==datetime.now().date().year]['page_title'].tolist()
curr_total_title_list

,page_url,page_title,records_total,pages_total,records_max,records_value,last_article_date
0,https://artinvestment.ru/invest/analytics/,Рынок искусства: аналитика и прогнозы,348,12,100,30,2022
1,https://artinvestment.ru/invest/ideas/,Инвестиционные идеи на рынке искусства,103,4,100,30,2022
2,https://artinvestment.ru/invest/collector/,Советы начинающим коллекционерам,19,1,100,30,2020
3,https://artinvestment.ru/invest/xxicentury/,21-й век,156,6,100,30,2022
4,https://artinvestment.ru/invest/artistofweek/,Художник недели,185,7,100,30,2021
5,https://artinvestment.ru/invest/law/,Искусство: экономика и право,32,2,100,30,2020
6,https://artinvestment.ru/invest/rating/,Рейтинги,92,4,100,30,2019
7,https://artinvestment.ru/invest/events/,Обзоры событий в мире искусства,163,6,100,30,2021
8,https://artinvestment.ru/invest/interviews/,Интервью,48,2,100,30,2020
9,https://artinvestment.ru/invest/stories/,Истории с искусством,15,1,100,30,2021


In [104]:
# new dict for current parsing
new_tree_dict = {}
for k, v in prob_for_pars_articles_dict.items():
  if v[1] in curr_total_title_list:
    new_tree_dict.update({int(k): prob_for_pars_articles_dict[k]})

new_tree_dict

{21: ['https://artinvestment.ru/invest/analytics/',
  'Рынок искусства: аналитика и прогнозы'],
 22: ['https://artinvestment.ru/invest/ideas/',
  'Инвестиционные идеи на рынке искусства'],
 24: ['https://artinvestment.ru/invest/xxicentury/', '21-й век'],
 37: ['https://artinvestment.ru/invest/first_names/', 'Первые имена'],
 39: ['https://artinvestment.ru/news/artnews/', 'Новости искусства'],
 40: ['https://artinvestment.ru/news/auctnews/',
  'Аукционы произведений искусства'],
 41: ['https://artinvestment.ru/news/exhibitions/',
  'Выставки произведений искусства']}

### Parsing dict updates

In [105]:
check_tree(artinvestment_tree, new_tree_dict)

Length of navigation three has changed - True
Changes in the old tree are: {1: ['https://auction.artinvestment.ru/', 'AI Аукцион'], 2: ['https://artinvestment.ru/services/form/?type=sell', 'Прием лотов'], 3: ['https://auction.artinvestment.ru/rules', 'Правила участия'], 4: ['https://artinvestment.ru/xxicentury/', 'Галерея AI "21-й век"'], 5: ['https://artinvestment.ru/about/', 'О проекте'], 6: ['https://artinvestment.ru/about/about_us/', 'О сайте'], 7: ['https://artinvestment.ru/about/council/', 'Экспертный совет'], 8: ['https://artinvestment.ru/about/risk_indicator/', 'Индикатор инвестиционного риска'], 9: ['https://artinvestment.ru/about/license/', 'Условия пользования сайтом'], 10: ['https://artinvestment.ru/subscription/', 'Подписка на сайте'], 11: ['https://artinvestment.ru/about/rss/', 'Лента новостей на Вашем сайте'], 12: ['https://artinvestment.ru/about/research/', 'Научная деятельность'], 13: ['https://artinvestment.ru/auctions/', 'Художники и цены'], 14: ['https://artinvestme

In [124]:
index_list = []
for k, v in new_tree_dict.items():
  index_list.append(int(k))

In [126]:
artinvest_datasets = [] # different structure
artinvest_news = [39, 40, 41]
# 'Новости искусства', 'Аукционы произведений искусства', 'Выставки произведений искусства'
artinvest_articles = list(set(index_list) - set(artinvest_news))
artinvest_others = [] # different structure

In [106]:
save_dict(new_tree_dict, artinvestment_tree) # 3.2 with the last navigation tree

## Contacts

In [16]:
contacts_page_url = 'https://artinvestment.ru/contacts/'
contacts_page = get_page(contacts_page_url)
contacts_page.title.string

'Контакты'

In [108]:
contact_tree = contacts_page.find('div', class_='info').find('div', class_='nav')

list_titles, list_links = get_topic_dict(contact_tree)
list_titles

['Условия пользования',
 'Обработка персональных данных',
 'Подписка',
 'Реклама',
 'Telegram',
 'Facebook',
 'Instagram',
 'Карта сайта',
 'AI Аукцион',
 'RSS',
 'Галерея Элизиум',
 'Галерея AI "21-й век"']

In [111]:
contacts_titles = ['Telegram', 'Facebook', 'Instagram']
contacts_links = []
for t in contacts_titles:
  contacts_links.append(list_links[list_titles.index(t)])

## Dict creating

In [113]:
links_and_titles = {'https://artinvestment.ru/': {'news': {'links': [new_tree_dict[x][0] for x in artinvest_news], 
                                                           'titles': [new_tree_dict[x][1] for x in artinvest_news]}, 
                                                  'articles': {'links': [new_tree_dict[x][0] for x in artinvest_articles], 
                                                               'titles': [new_tree_dict[x][1] for x in artinvest_articles]}, 
                                                  'datasets': {'links': [new_tree_dict[x][0] for x in artinvest_datasets], 
                                                               'titles': [new_tree_dict[x][1] for x in artinvest_datasets]}, 
                                                  'others': {'links': [new_tree_dict[x][0] for x in artinvest_others], 
                                                             'titles': [new_tree_dict[x][1] for x in artinvest_others]},
                                                  'contacts': {'links': [contacts_links], 
                                                               'titles': [contacts_titles]}
                                                  }
                     }

links_and_titles

{'https://artinvestment.ru/': {'articles': {'links': ['https://artinvestment.ru/invest/analytics/',
    'https://artinvestment.ru/invest/ideas/',
    'https://artinvestment.ru/invest/xxicentury/',
    'https://artinvestment.ru/invest/first_names/'],
   'titles': ['Рынок искусства: аналитика и прогнозы',
    'Инвестиционные идеи на рынке искусства',
    '21-й век',
    'Первые имена']},
  'contacts': {'links': [['https://t.me/artinvestmentru',
     'https://www.facebook.com/artinvestment/',
     'https://www.instagram.com/artinvestment_ru/']],
   'titles': [['Telegram', 'Facebook', 'Instagram']]},
  'datasets': {'links': [], 'titles': []},
  'news': {'links': ['https://artinvestment.ru/news/artnews/',
    'https://artinvestment.ru/news/auctnews/',
    'https://artinvestment.ru/news/exhibitions/'],
   'titles': ['Новости искусства',
    'Аукционы произведений искусства',
    'Выставки произведений искусства']},
  'others': {'links': [], 'titles': []}}}

In [ ]:
# save_parsing_dict(links_and_titles) # 3.1 Save dict with sites for parsing

## Dict updating

In [ ]:
sites_for_parsing = open_parsing_dict() # 4.1 Open dict with sites for parsing
sites_for_parsing.update(links_and_titles)

## Художники и цены

In [ ]:
# Художники и цены
url = list_links[0]

start_time = time.time()
response = requests.get(url, headers={'User-Agent': 'Chrome/97.0.4692.71'}) # timeout=0.001
# Edge/97.0.1072.55
# Chrome/97.0.4692.71
for key, value in tree_dict.items():
  if key==url:
    print(value)
print(response.status_code)
print(response.encoding)
print(response.raise_for_status())
print("--- %s seconds ---" % (time.time() - start_time))

Художники и цены
200
UTF-8
None
--- 1.8451573848724365 seconds ---


In [ ]:
artists_page = BeautifulSoup(response.text, 'html.parser')
#print(artists_page.prettify())

<!DOCTYPE html>
<html lang="ru" prefix="og: http://ogp.me/ns#" xml:lang="ru">
 <head>
  <!-- Global site tag (gtag.js) - Google Analytics -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-66149674-1">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
        function gtag(){dataLayer.push(arguments);}
        gtag('js', new Date());

        gtag('config', 'UA-66149674-1', { 'optimize_id': 'GTM-T4V5FRP'});

        gtag('config', 'AW-474414610');
  </script>
  <!-- Anti-flicker snippet (recommended)  -->
  <style>
   .async-hide { opacity: 0 !important}
  </style>
  <script>
   (function(a,s,y,n,c,h,i,d,e){s.className+=' '+y;h.start=1*new Date;
        h.end=i=function(){s.className=s.className.replace(RegExp(' ?'+y),'')};
        (a[n]=a[n]||[]).hide=h;setTimeout(function(){i();h.end=null},c);h.timeout=c;
    })(window,document.documentElement,'async-hide','dataLayer',4000,
        {'GTM-T4V5FRP':true});
  </script>
  <title>
   Художники и 

In [ ]:
table_blocks = artists_page.find_all('div', class_='pager')
# div class="artists-list"
len(table_blocks)

2

In [ ]:
table_blocks[0]

<div class="pager">
<div class="table"><div>
<div>Записей: <em>14888</em></div>
<div>Страниц: <em>497</em></div>
<div>На стр <input id="page" name="citems" ss="https://artinvestment.ru/auctions/?citems=" title="Max 200" type="text" value="30"/></div>
<div class="pages">
<div class="table"><div>
<div class="first"><a href="https://artinvestment.ru/auctions/?page=1&amp;citems=30" title="Первая">Первая</a></div>
<div class="numbers"><ul>
<li class="current"><a href="https://artinvestment.ru/auctions/">1</a></li>
<li><a href="https://artinvestment.ru/auctions/?page=2&amp;citems=30">2</a></li> <li><a href="https://artinvestment.ru/auctions/?page=3&amp;citems=30">3</a></li> </ul></div>
<div class="next"><a href="https://artinvestment.ru/auctions/?page=2&amp;citems=30">»</a></div>
<div class="last"><a href="https://artinvestment.ru/auctions/?page=497&amp;citems=30" title="Последняя">Последняя</a></div>
</div></div>
</div>
</div></div>
</div>

In [ ]:
artinvest_artists_stat = pd.DataFrame({'page_url': [list_links[0]],
                                       'page_title': [list_titles[0]],
                                       'records_total': [int(table_blocks[0].find_all('em')[0].text)],
                                       'pages_total': [int(table_blocks[0].find_all('em')[1].text)],
                                       'records_max': [int(table_blocks[0].find('input').get('title').split(' ')[1])],
                                       'records_value': [int(table_blocks[0].find('input').get('value'))],
                                       'first_page_url': [table_blocks[0].find('a').get('href')],
                                       'parsing_date': [datetime.now().date()],
                                       'parsing_time': [datetime.now().time()]})
artinvest_artists_stat

,page_url,page_title,records_total,pages_total,records_max,records_value,first_page_url,parsing_date,parsing_time
0,https://artinvestment.ru/auctions/,Художники и цены,14888,497,200,30,https://artinvestment.ru/auctions/?page=1&cite...,2022-01-11,16:56:51.181806


# theartnewspaper.ru
- nav tree contains only articles' srctions, nothing else 

## Navigation tree

In [22]:
# Collect page data
theartnewspaper_url = urls_news[0]
theartnewspaper_page = get_page(theartnewspaper_url) # 1. Collect request status
theartnewspaper_page.title.string

'The Art Newspaper Russia — новости искусства'

In [23]:
# Collect topics for parsing

def get_topic_dict_tanp(bs_page):
  list_titles = []
  list_links = []
  #count_list = []
  #count = 1
  for mmr in bs_page.find('div', class_='mainMenuRoot').find_all('a'):
    if mmr.get_text() is not None:
      if str(mmr.get_text()) not in list_titles:
        list_titles.append(str(mmr.get_text()))
        #count_list.append(count)
        #count = count+1
        list_links.append(theartnewspaper_url[:-1] + str(mmr.get('href')))

  for gh in bs_page.find_all('div', class_='groupHeader'):
    for add in gh.find_all('a'):
      if add.get_text() is not None:
        if str(add.get_text()) not in list_titles:
          list_titles.append(str(add.get_text()))
          #count_list.append(count)
          #count = count+1
          list_links.append(theartnewspaper_url[:-1] + str(add.get('href')))

  #tree_dict = {key: value for key, value in zip(count_list, list_links)}
  return list_titles, list_links #tree_dict

In [ ]:
#new_tree_dict = get_topic_dict_tanp(theartnewspaper_page) # Collect topics for parsing
#save_dict(new_tree_dict, theartnewspaper_tree) # 3.2 with the last navigation tree

In [24]:
# Collect topics for parsing from navigation tree
curr_list_titles, curr_list_links = get_topic_dict_tanp(theartnewspaper_page)
print('curr_list_titles length is {}'.format(len(curr_list_titles)))

# sections that are not in the navigation tree (don't have links => manual collecting)
page_title_list = []
for p_title in theartnewspaper_page.find_all('div', class_='postPreviewsItemSection'):
  page_title_list.append(p_title.get_text())

add_title_list = list(set(page_title_list) - set(curr_list_titles))

curr_total_title_list = curr_list_titles + add_title_list
print('curr_total_title_list length is {}'.format(len(curr_total_title_list)))

# finding what is not in current total tree dict
tanp_curr_total_tree = open_dict(tanp_total_tree)
print('tanp_curr_total_tree length is {}'.format(len(tanp_curr_total_tree)))

tanp_total_list_old = []
for k, v in tanp_curr_total_tree.items():
  tanp_total_list_old.append(v[1])

new_not_in_old_list = list(set(curr_total_title_list) - set(tanp_total_list_old))
print('count of titles that is not in the current total tree is {}'.format(len(new_not_in_old_list)))

count_list = []
count = len(tanp_curr_total_tree)
for t in new_not_in_old_list:
  count += 1
  count_list.append(count)

add_links_list = []
for t in new_not_in_old_list:
  if t in curr_list_titles:
    add_links_list.append(curr_list_links[curr_list_titles.index(t)])
  else:
    add_links_list.append('-')

list_links = []
for l in add_links_list:
  list_links.append([l, new_not_in_old_list[add_links_list.index(l)]])

add_dict = {key: value for key, value in zip(count_list, list_links)}
add_dict

curr_list_titles length is 10
curr_total_title_list length is 15
tanp_curr_total_tree length is 58
count of titles that is not in the current total tree is 0


{}

In [ ]:
# making corrections if it's needed
add_dict.update({56: ['https://www.theartnewspaper.ru/sections/premii-i-nagrady/', 'Премии и награды']})
add_dict

{55: ['https://www.theartnewspaper.ru/sections/guerlain/', 'Guerlain'],
 56: ['https://www.theartnewspaper.ru/sections/premii-i-nagrady/',
  'Премии и награды'],
 57: ['https://www.theartnewspaper.ru/sections/gallery/', 'Галереи'],
 58: ['https://www.theartnewspaper.ru/sections/video/', 'Видео']}

In [ ]:
# updating the current total tree
tanp_curr_total_tree.update(add_dict)
len(tanp_curr_total_tree)

58

In [25]:
# saving total tree dict
save_dict(tanp_curr_total_tree, tanp_total_tree) # 3.2 with the last navigation tree

In [26]:
# new dict for current parsing
new_tree_dict = {}
for k, v in tanp_curr_total_tree.items():
  if v[1] in curr_total_title_list:
    new_tree_dict.update({int(k): tanp_curr_total_tree[k]})

new_tree_dict

{1: ['https://www.theartnewspaper.ru/sections/news/', 'Новости'],
 7: ['https://www.theartnewspaper.ru/sections/luxury/', 'Роскошь'],
 8: ['https://www.theartnewspaper.ru/sections/interviu/', 'Интервью'],
 12: ['https://www.theartnewspaper.ru/sections/shows/', 'Выставки'],
 16: ['https://www.theartnewspaper.ru/sections/markets/', 'Арт-рынок'],
 22: ['https://www.theartnewspaper.ru/sections/conservation/', 'Реставрация'],
 24: ['https://www.theartnewspaper.ru/sections/Premiia-The-Art-Newspaper-Russia/',
  'Премия The Art Newspaper Russia'],
 31: ['https://www.theartnewspaper.ru/sections/mneniia/', 'Мнения'],
 36: ['https://www.theartnewspaper.ru/sections/Partnerstvo/', 'Промо'],
 38: ['https://www.theartnewspaper.ru/sections/books/', 'Книги'],
 48: ['https://www.theartnewspaper.ru/sections/samara-rossiia/',
  'Самара Россия'],
 55: ['https://www.theartnewspaper.ru/sections/guerlain/', 'Guerlain'],
 56: ['https://www.theartnewspaper.ru/sections/premii-i-nagrady/',
  'Премии и награды'],


### Parsing dict updates

In [27]:
check_tree(theartnewspaper_tree, new_tree_dict)

Length of navigation three has changed - False
Changes in the old tree are: {35: ['https://www.theartnewspaper.ru/sections/los-andzheles-ssha/', 'Лос-Анджелес США']}
Changes in the new tree are: {31: ['https://www.theartnewspaper.ru/sections/mneniia/', 'Мнения']}


In [ ]:
index_list = []
for k, v in new_tree_dict.items():
  index_list.append(int(k))

In [28]:
# 9 and 10 are excluded
theartnewspaper_datasets = []  # different structure
theartnewspaper_news = [1, 12, 16]
# 'Новости', 'Выставки', 'Арт-рынок'
theartnewspaper_others = [57, 58]  # different structure
# 'Галереи', 'Видео'
theartnewspaper_articles = list(set(index_list) - set(theartnewspaper_news) - set(theartnewspaper_others))

In [29]:
# saving current tree dict
save_dict(new_tree_dict, theartnewspaper_tree) # 3.2 with the last navigation tree

## Contacts

In [114]:
contacts = theartnewspaper_page.find('div', class_='preFooterRoot').find('div', class_='leftCol')
list_links = []
for a in contacts.find_all('a'):
  list_links.append(a.get('href'))

list_links

['https://www.facebook.com/TheArtNewspaperRussia',
 'https://www.instagram.com/theartnewspaperrussia/',
 'https://vk.com/theartnewspaper.russia',
 'https://t.me/TANR_official',
 'https://ok.ru/theartnewspaperrussia/',
 'https://twitter.com/TANRussia',
 'https://www.youtube.com/c/TheArtNewspaperRussia']

In [121]:
index_list = [3, 0, 1]
contacts_titles = ['Telegram', 'Facebook', 'Instagram']
contacts_links = []
for i in index_list:
  contacts_links.append(list_links[i])

## Dict creating

In [123]:
links_and_titles = {'https://www.theartnewspaper.ru/': {'news': {'links': [new_tree_dict[x][0] for x in theartnewspaper_news], 
                                                                 'titles': [new_tree_dict[x][1] for x in theartnewspaper_news]}, 
                                                        'articles': {'links': [new_tree_dict[x][0] for x in theartnewspaper_articles], 
                                                                     'titles': [new_tree_dict[x][1] for x in theartnewspaper_articles]},
                                                        'datasets': {'links': [new_tree_dict[x][0] for x in theartnewspaper_datasets], 
                                                                     'titles': [new_tree_dict[x][1] for x in theartnewspaper_datasets]}, 
                                                        'others': {'links': [new_tree_dict[x][0] for x in theartnewspaper_others], 
                                                                   'titles': [new_tree_dict[x][1] for x in theartnewspaper_others]},
                                                        'contacts': {'links': [contacts_links],
                                                                     'titles': [contacts_titles]}
                                                        }
                         }
links_and_titles

KeyError: ignored

## Dict updating

In [31]:
sites_for_parsing = open_parsing_dict() # 4.1 Open dict with sites for parsing
sites_for_parsing.update(links_and_titles)

In [32]:
len(sites_for_parsing)

2

In [33]:
sites_for_parsing

{'https://artinvestment.ru/': {'articles': {'links': ['https://artinvestment.ru/invest/analytics/',
    'https://artinvestment.ru/invest/xxicentury/',
    'https://artinvestment.ru/invest/events/',
    'https://artinvestment.ru/invest/painters/'],
   'titles': ['Рынок искусства: аналитика и прогнозы',
    '21-й век',
    'Обзоры событий в мире искусства',
    'Рассказы о художниках']},
  'contacts': {'links': ['https://t.me/artinvestmentru',
    'https://www.facebook.com/artinvestment/',
    'https://www.instagram.com/artinvestment_ru/'],
   'titles': ['Telegram', 'Facebook', 'Instagram']},
  'datasets': {'links': ['https://artinvestment.ru/auctions/',
    'https://artinvestment.ru/auctions/auclist/auclist.html'],
   'titles': ['Художники и цены', 'Аукционы']},
  'news': {'links': ['https://artinvestment.ru/news/artnews/',
    'https://artinvestment.ru/news/auctnews/',
    'https://artinvestment.ru/news/exhibitions/'],
   'titles': ['Новости искусства',
    'Аукционы произведений искус

In [34]:
save_parsing_dict(sites_for_parsing) # 3.1 Save dict with sites for parsing